Объявление нужных нам модулей и классов.

In [1]:
from random import randint
from typing import List

import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt

Создание DataFrame с именованием колонок.

In [ ]:
df_start = pd.read_csv("annotation.csv", sep=" ", encoding="windows-1251", header=None)
my_series = pd.DataFrame({
    "class": df_start.iloc[:, 2],
    "abspath": df_start.iloc[:, 0]
})

my_series

Добавление колонки label, хранящая метку класса(0 для котов, 1 для собак).

In [ ]:
my_series["label"] = 0

for i, class_ in enumerate(my_series["class"]):
    if class_ == "dog":
        my_series.iloc[[i], [2]] = 1

my_series

Добавление колонок с шириной, высотой и глубиной каждого изображения.

In [ ]:
my_series["height"] = 0
my_series["width"] = 0
my_series["depth"] = 0
my_series["pixels"] = 0

for i, path in enumerate(my_series["abspath"]):
    data = cv2.imread(path[-20:])
    my_series.iloc[[i], [3]] = data.shape[0]
    my_series.iloc[[i], [4]] = data.shape[1]
    my_series.iloc[[i], [5]] = data.shape[2]
    my_series.iloc[[i], [6]] = data.size

my_series

Статистические данные о DataFrame(данные сбалансированы).

In [ ]:
my_series.describe()

Функция, возвращающая DataFrame, отфильтрованная по метке класса.

In [12]:
def part_5(my_series: pd.DataFrame, label: int) -> pd.DataFrame:
    '''Returns a DataFrame with the given label'''
    return my_series[my_series["label"] == label]

label_my_series = part_5(my_series, 1)

Функция, возвращающая отфильтрованный DataFrame по метке, высоте и ширине.

In [13]:
def part_6(my_series: pd.DataFrame, label: int, h: int, w: int) -> pd.DataFrame:
    '''Returns a DataFrame with the given label and appropriate dimensions'''
    tmp_1 = my_series[my_series["label"] == label]
    tmp_2 = tmp_1[tmp_1["height"] <= h]
    return tmp_2[tmp_2["width"] <= w]

image_my_series = part_6(my_series, 0, 319, 500)

Функция, вычисляющая кол-во пикселей для исходного DataFrame и макс., мин. и ср. значения пикселей для DataFrame, отфильтрованного по метке.

In [ ]:
def part_7(label: int) -> None:
    '''Outputs the minimum, maximum, and median pixels'''
    pix_my_series = part_5(my_series, label)
    
    print("The maximum number of pixels is", pix_my_series["pixels"].max())
    print("The minimum number of pixels is", pix_my_series["pixels"].min())
    print("The average number of pixels is", pix_my_series["pixels"].mean())

part_7(0)

Функция, строящая гистограмму.

In [10]:
def part_8(my_series: pd.DataFrame, label: int) -> List[np.ndarray]:
    '''Returns 3 arrays with histograms for each channel'''
    label_series = my_series[my_series["label"] == label]
    rand_num = randint(0, len(label_series["abspath"]))
    abspath = my_series.iloc[rand_num, 1]

    image = cv2.imread(abspath[-20:])

    histr_1 = cv2.calcHist([image], [0], None, [256], [0,256])
    histr_2 = cv2.calcHist([image], [1], None, [256], [0,256])
    histr_3 = cv2.calcHist([image], [2], None, [256], [0,256])
    
    return [histr_1, histr_2, histr_3]

Отрисовка гистограмм.

In [ ]:
histr_1, histr_2, histr_3 = part_8(my_series, 0)
col = ('b','g','r')
plt.plot(histr_1, color=col[0])
plt.plot(histr_2, color=col[1])
plt.plot(histr_3, color=col[2])
plt.xlabel("value")
plt.ylabel("count")
plt.title("histogram")
plt.xlim([0,256])
plt.show()

Создание электронной таблицы, куда загружаем весь наш исходный DataFrame.

In [59]:
my_series.to_csv("my_series.csv", sep=" ")